In [ ]:
import pandas as pd
import numpy as np
import json
import requests

In [ ]:
df1 = pd.DataFrame()
row = {}
l1 = ['IsCrossListed', 'PublishedCourseID', 'ScheduledCourseID', 'prefix', 'number', 'title', 'description', 'units', 'suffix', 'restriction_by_class', 'restriction_by_school', 'restriction_by_major', 'coreq_text', 'prereq_text', 'sequence']
l2 = ['id', 'session', 'dclass_code', 'title', 'decription', 'notes', 'type', 'units', 'spaces_available', 'number_registered', 'wait_qty', 'canceled', 'blackboard', 'day', 'start_time', 'end_time', 'location', 'IsDistanceLearning', 'section_title']
l3 = ['last_name', 'first_name', 'bio_url']
l4 = ['format', 'filesize']
appended_data = []

def scrape():
    df = pd.read_excel('urllist.xlsx')
    url_list = df["url"].tolist()
#     url_list1 = [url_list[0]]
#     print(url_list)
#     i = 1
    for i, url in enumerate(url_list, start=1):
#         print(url)
#         if i == 100 or i == 200:
#             input1 = input("DO you want to continue?")
#             if input1 == "y":
#                 pass
#             else:
#                 break
#         else:
#             pass
        
        response = requests.get(url)
        if response.status_code == 200:  # OK status code
            json_text = response.content.decode('utf-8')
            json_obj = json.loads(json_text)
        else:
            print('Error: HTTP Status Code', response.status_code)
            input1 = input("DO you want to Quit or wait")

        if "course" in json_obj.get("OfferedCourses", np.nan):
            if type(json_obj["OfferedCourses"]["course"]) == dict:
                type1 = "dict"
                course = json_obj["OfferedCourses"]["course"]
                abcd(course)
                
            elif type(json_obj["OfferedCourses"]["course"]) == list:
                type1 = "list"
                for course in json_obj["OfferedCourses"]["course"]:
                    abcd(course)

            else:
                print("NEITHER DICTIONARY NOR LIST")
                continue

        appended_data.append(df1)
        print(f'''type["course"]: {type1}   |  URL DONE: {i}  |   {url}''')

def abcd(course):
    global df1
    global row
    global l1
    global l2
    global l3
    global l4 
    for i in l1[0:3]:
        if type(course.get(i, np.nan)) != list:
            row[i] = course.get(i, np.nan)
        elif type(course.get(i, np.nan)) == list:
            a = len(course.get(i, np.nan))
            for j in range(a):
                row[f'{i}/{j}'] = course.get(i, np.nan)[j]

    for i in l1[3:]:
        if type(course.get('CourseData', np.nan)[i]) != list:
            row[f'CourseData/{i}'] = course.get('CourseData', np.nan)[i]
        elif type(course.get('CourseData', np.nan)[i]) == list:
            a = len(course.get('CourseData', np.nan)[i])
            for j in range(a):
                row[f'CourseData/{i}/{j}'] = course.get('CourseData', np.nan)[i][j]

    SectionData = course['CourseData']['SectionData']
    if type(SectionData) == dict:
        for i in l2:
            if type(SectionData.get(i, np.nan)) != list:
                row[f'CourseData/SectionData/0/{i}'] = SectionData.get(i, np.nan)                        
            elif type(SectionData.get(i, np.nan)) == list:
                a = len(SectionData.get(i, np.nan))
                for j in range(a):
                    row[f'CourseData/SectionData/0/{i}/{j}'] = SectionData.get(i, np.nan)[j]                        

        if 'instructor' in SectionData:
            if type(SectionData.get('instructor', np.nan)) == dict:
                for i in l3:
                    if type(SectionData['instructor'].get(i,np.nan)) != list:
                        row[f'CourseData/SectionData/0/instructor/0/{i}'] = SectionData['instructor'].get(i,np.nan)
                    elif type(SectionData['instructor'].get(i,np.nan)) == list:
                        a = len(SectionData['instructor'].get(i,np.nan))
                        for j in range(a):
                            row[f'CourseData/SectionData/0/instructor/0/{i}/{j}'] = SectionData['instructor'].get(i,np.nan)[j]


            elif type(SectionData.get('instructor', np.nan)) == list:
                inst_length = len(SectionData.get('instructor', np.nan))
                for inst in range(inst_length):
                    for i in l3:
                        if type(SectionData['instructor'][inst].get(i,np.nan)) != list:
                            row[f'CourseData/SectionData/0/instructor/{inst}/{i}'] = SectionData['instructor'][inst].get(i,np.nan)
                        elif type(SectionData['instructor'][inst].get(i,np.nan)) == list:
                            a = len(SectionData['instructor'][inst].get(i,np.nan))
                            for j in range(a):
                                row[f'CourseData/SectionData/0/instructor/{inst}/{i}/{j}'] = SectionData['instructor'][inst].get(i,np.nan)[j]

        if 'syllabus' in SectionData:
            if type(SectionData.get('syllabus', np.nan)) == dict:
                for i in l4:
                    if type(SectionData['syllabus'].get(i,np.nan)) != list:
                        row[f'CourseData/SectionData/0/syllabus/0/{i}'] = SectionData['syllabus'].get(i,np.nan)
                    elif type(SectionData['syllabus'].get(i,np.nan)) == list:
                        a = len(SectionData['syllabus/{j}'].get(i,np.nan))[j]
                        for j in range(a):
                            row[f'CourseData/SectionData/0/syllabus/0/{i}'] = SectionData['syllabus'].get(i,np.nan)
            elif type(SectionData.get('syllabus', np.nan)) == list:
                inst_length = len(SectionData.get('syllabus', np.nan))
                for inst in range(inst_length):
                    for i in l4:
                        if type(SectionData['syllabus'][inst].get(i,np.nan)) != list:
                            row[f'CourseData/SectionData/0/syllabus/{inst}/{i}'] = SectionData['syllabus'][inst].get(i,np.nan)
                        elif type(SectionData['syllabus'][inst].get(i,np.nan)) == list:
                            a = len(SectionData['syllabus'][inst].get(i,np.nan))
                            for j in range(a):
                                row[f'CourseData/SectionData/0/syllabus/{inst}/{i}/{j}'] = SectionData['syllabus'][inst].get(i,np.nan)[j]

    elif type(SectionData) == list:
        section_length = len(SectionData)
        for sl in range(section_length):
            for i in l2:
                if type(SectionData[sl].get(i, np.nan)) != list:
                    row[f'CourseData/SectionData/{sl}/{i}'] = SectionData[sl].get(i, np.nan)
                elif type(SectionData[sl].get(i, np.nan)) == list:
                    a = len(SectionData[sl].get(i, np.nan))
                    for j in range(a):
                        row[f'CourseData/SectionData/{sl}/{i}/{j}'] = SectionData[sl].get(i, np.nan)[j]

            if 'instructor' in SectionData[sl]:
                if type(SectionData[sl].get('instructor', np.nan)) == dict:
                    for i in l3:
                        if type(SectionData[sl]['instructor'].get(i,np.nan)) != list:
                            row[f'CourseData/SectionData/{sl}/instructor/0/{i}'] = SectionData[sl]['instructor'].get(i,np.nan)
                        elif type(SectionData[sl]['instructor'].get(i,np.nan)) == list:
                            a = len(SectionData[sl]['instructor'].get(i,np.nan))
                            for j in range(a):
                                row[f'CourseData/SectionData/{sl}/instructor/0/{i}/{j}'] = SectionData[sl]['instructor'].get(i,np.nan)[j]
                elif type(SectionData[sl].get('instructor', np.nan)) == list:
                    inst_length = len(SectionData[sl].get('instructor', np.nan))
                    for inst in range(inst_length):
                        for i in l3:
                            if type(SectionData[sl]['instructor'][inst].get(i,np.nan)) != list:
                                row[f'CourseData/SectionData/{sl}/instructor/0/{inst}/{i}'] = SectionData[sl]['instructor'][inst].get(i,np.nan)
                            elif type(SectionData[sl]['instructor'][inst].get(i,np.nan)) == list:
                                a = len(SectionData[sl]['instructor'][inst].get(i,np.nan))
                                for j in range(a):
                                    row[f'CourseData/SectionData/{sl}/instructor/0/{inst}/{i}/{j}'] = SectionData[sl]['instructor'][inst].get(i,np.nan)[j]

            if 'syllabus' in SectionData[sl]:
                if type(SectionData[sl].get('syllabus', np.nan)) == dict:
                    for i in l4:
                        if type(SectionData[sl]['syllabus'].get(i,np.nan)) != list:
                            row[f'CourseData/SectionData/{sl}/syllabus/0/{i}'] = SectionData[sl]['syllabus'].get(i,np.nan)
                        elif type(SectionData[sl]['syllabus'].get(i,np.nan)) == list:
                            a = len(SectionData[sl]['syllabus'].get(i,np.nan))
                            for j in range(a):
                                row[f'CourseData/SectionData/{sl}/syllabus/0/{i}/{j}'] = SectionData[sl]['syllabus'].get(i,np.nan)[j]

                elif type(SectionData[sl].get('syllabus', np.nan)) == list:
                    inst_length = len(SectionData[sl].get('syllabus', np.nan))
                    for inst in range(inst_length):
                        for i in l4:
                            if type(SectionData[sl]['syllabus'][inst].get(i,np.nan)) != list:
                                row[f'CourseData/SectionData/{sl}/syllabus/0/{inst}/{i}'] = SectionData[sl]['syllabus'][inst].get(i,np.nan)
                            elif type(SectionData[sl]['syllabus'][inst].get(i,np.nan)) == list:
                                a = len(SectionData[sl]['syllabus'][inst].get(i,np.nan))
                                for j in range(a):
                                    row[f'CourseData/SectionData/{sl}/syllabus/0/{inst}/{i}/{j}'] = SectionData[sl]['syllabus'][inst].get(i,np.nan)[j]

    df2 = pd.DataFrame([row], index=None)
    df1 = pd.concat([df1,df2], axis=0, ignore_index=True)



In [ ]:
scrape()

In [ ]:
# To check: How much url's Dataframe are collected in the appended_data list
len(appended_data)

In [ ]:
df6 = pd.concat(appended_data, ignore_index=True)
#To remove any dictionay type value {} in the DataFrame.
df6 = df6.applymap(lambda x: np.nan if isinstance(x, dict) else x)
df6

In [ ]:
# To remove/drop empty or null columns from the DataFrame
df6 = df6.dropna(axis=1, how='all')
df6

In [ ]:
df6.to_csv("usc_final2.csv", index=False)

# META CODE
### To check any information in the dataframe. You can move below cells up and down accordingly to your own requirements.

In [ ]:
appended_data1 = appended_data[150]
appended_data1.info(verbose=False, memory_usage="deep")
# appended_data1

In [ ]:
s = df6["CourseData/SectbionData/137/instructor/0/first_name"]
if s.notnull().any():
    print(s[s.notnull()])

In [ ]:
from ast import literal_eval

In [ ]:
len(json_obj["OfferedCourses"]["course"])